In [1]:
import numpy as np
import pandas as pd
import itertools
from numba import jit, prange
import json

dfa = pd.read_csv(r'heart2.csv')
dfa["Sex2"] = np.nan
dfa["ChestPainType2"] = np.nan

for i in range(dfa.shape[0]):
    if dfa["Sex"][i] == "M":
        dfa["Sex2"][i] = 0
    else:
        dfa["Sex2"][i] = 1

    if dfa["ChestPainType"][i] == "ASY":
        dfa["ChestPainType2"][i] = 0
    elif dfa["ChestPainType"][i] == "NAP":
        dfa["ChestPainType2"][i] = 1
    elif dfa["ChestPainType"][i] == "NAP":
        dfa["ChestPainType2"][i] = 2
    else:
        dfa["ChestPainType2"][i] = 3

# dfa = dfa[['Age', 'RestingBP', 'MaxHR', 'HeartDisease']].copy()

dfb = dfa[dfa['RestingBP'] > 0].copy()
df = dfb[['RestingBP', 'HeartDisease']].copy()
# df = dfa[dfa['RestingBP'] > 0].copy()

catvalues = [3, 2]

# G1 = [[i+1 for i in range(df.shape[1])]]
G1 = [[1],[2]]
E = [[i,j] for i in range(1, df.shape[1]) for j in range(i+1, df.shape[1]+1)]

# Creating design matrix A and converting contingency table u into vector of counts

df2 = df.copy()
n = df.shape[0]

print("n =", n)

for i in range(len(list(df.columns))):
    dfmin = df[list(df.columns)[i]].min()
    dfmax = df[list(df.columns)[i]].max()
    dfrange = (dfmax - dfmin)/catvalues[i]
#     if catvalues[i] > 2:
#         for j in range(n):
#             jval = df[list(df.columns)[i]][j]
# #             cond = [dfmin + k*dfrange <= jval < dfmin + (k+1)*dfrange if dfmin <= k <= (catvalues[k]-2) else dfmin + k*dfrange <= jval <= dfmin + (k+1)*dfrange for k in range(catvalues[i])]
#             cond = [dfmin + k*dfrange <= jval < dfmin + (k+1)*dfrange if k <= catvalues[i]-2 else dfmin + k*dfrange <= jval <= dfmin + (k+1)*dfrange for k in range(catvalues[i])]
#             df2[list(df.columns)[i]][j] = cond.index(True)
            
    for j in df.index.values.tolist():
        jval = df[list(df.columns)[i]][j]
#             cond = [dfmin + k*dfrange <= jval < dfmin + (k+1)*dfrange if dfmin <= k <= (catvalues[k]-2) else dfmin + k*dfrange <= jval <= dfmin + (k+1)*dfrange for k in range(catvalues[i])]
        cond = [dfmin + k*dfrange <= jval < dfmin + (k+1)*dfrange if k <= catvalues[i]-2 else dfmin + k*dfrange <= jval <= dfmin + (k+1)*dfrange for k in range(catvalues[i])]
        df2[list(df.columns)[i]][j] = cond.index(True)
            
            
#     else:
#         for j in range(size):
#             jval = df[list(df.columns)[i]][j]
#             cond = [dfmin + i*dfrange <= jval < dfmin + (i+1)*dfrange if dfmin <= i <= (catvalues[i]-2) else dfmin + i*dfrange <= jval <= dfmin + (i+1)*dfrange for i in range(catvalues[i])]
#             df2[list(df.columns)[i]][j] = cond.index(True)

# a,b,c = 0,0,1
idx = pd.IndexSlice

def func(lst):
#     dictvalues = {"Pregnancies": 3, "Glucose": 3, "BloodPressure": 3, "SkinThickness": 3, "Insulin": 3, "BMI": 3, "DiabetesPedigreeFunction":3, "Age":3, "Outcome": 2}
#     orderdict = {"Pregnancies": 1, "Glucose": 2, "BloodPressure": 3, "SkinThickness":4, "Insulin":5, "BMI": 6, "DiabetesPedigreeFunction": 7, "Age":  8, "Outcome": 9}
#     order2dict = {1: "Pregnancies", 2: "Glucose", 3: "BloodPressure", 4: "SkinThickness", 5: "Insulin", 6: "BMI", 7: "DiabetesPedigreeFunction", 8: "Age", 9: "Outcome"}

    dictvalues = dict([(df.columns[i], catvalues[i]) for i in range(len(df.columns))])
    orderdict = dict([(df.columns[i], i+1) for i in range(len(df.columns))])
    order2dict = dict([(i+1, df.columns[i]) for i in range(len(df.columns))])
    
    unio = list(set().union(*lst))
    unio2 = sorted(unio)
    mydict = dict(zip(unio2, list(range(len(unio2)))))

    templ = np.zeros(len(unio))
    lst2 = [list(range(1, dictvalues[order2dict[i]]+1)) for i in unio2]
    lst3 = list(itertools.product(*lst2))
    templ = np.zeros(len(lst3))
    emptylist = []

    # Creating the design matrix
    for l in lst:
        for j in list(itertools.product(*[list(range(1, dictvalues[order2dict[i]]+1)) for i in sorted(l)])):
            for k in range(len(lst3)):
                if list(j) == [lst3[k][m] for m in [mydict[n] for n in sorted(l)]]: 
                    templ[k] = 1

            emptylist.append(templ)
            templ = np.zeros(len(lst3))
    
    ltest0 = [i for i in unio2]
    ltest = [df2[j] for j in [order2dict[i] for i in unio2]]
    data_crosstab = pd.crosstab(ltest[:-1], ltest[-1], margins = False, dropna = False) # set margins = True to see margins

    return np.array(emptylist), data_crosstab



script_4828146019857549440:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
script_4828146019857549440:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
script_4828146019857549440:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
script_4828146019857549440:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

n = 917


In [2]:
matrixA, utemp = func(lst = G1)

print(matrixA)
print(utemp)

matrixA1 = ["{" + str(list(x))[1:-1] + "}" for x in matrixA]
matrixA2 = "{" + str(matrixA1)[1:-1] + "}"
matrixA3 = "{" + str(matrixA)[1:-1] + "}"

tempstr = "matrix {"
for i in range(len(matrixA)-1):
    tempstr = tempstr + "{" + str(list(matrixA[i]))[1:-1] + "}" + ", "
    
tempstr = tempstr + "{" + str(list(matrixA[len(matrixA)-1]))[1:-1] + "}}"

with open("/Users/jc/readme2", 'w') as f2:
    f2.write(tempstr)
    
f2.close()


[[1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [1. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 1.]]
HeartDisease    0    1
RestingBP             
0              73   87
1             309  355
2              28   65


In [3]:
# Calculating rank of design matrix A

rankModel = np.linalg.matrix_rank(matrixA, tol=None, hermitian=False)
# rankModel = np.linalg.matrix_rank(testA1, tol=None, hermitian=False)
print("rankModel =", rankModel)



rankModel = 4


In [4]:
f3 = open("/Users/jc/readme2", "r")
# print(f3.read())

In [5]:
setup();
restart;
loadPackage "FourTiTwo"
A = value get "/Users/jc/readme2"
markovBasis = toricMarkov A
print(markovBasis)
"readme5" << toString markovBasis << close
copyFile("readme5", "/Users/jc/readme5")

--initialization text already in file: ~/.profile-Macaulay2
--initialization text already in file: ~/.login-Macaulay2
--initialization text already in file, no changes needed: /Users/jc/.bash_profile
--initialization text already in file, no changes needed: /Users/jc/.zprofile
--initialization text already in file, no changes needed: /Users/jc/.login
--initialization text already in file: ~/.emacs-Macaulay2
--initialization text already in file, no changes needed: /Users/jc/.emacs


| 0 0  1  -1 -1 1 |
| 1 -1 -1 1  0  0 |
| 1 -1 0  0  -1 1 |



In [6]:
pythonf = open("/Users/jc/readme5", "r")
x = pythonf.read()

# print(G1)

# print(markovBasis)
# restart;
# print(A)
# // "readme5" << toString markovBasis << close

In [7]:
if G1 != [[i+1 for i in range(df.shape[1])]]:
    new_string = x[7:].replace("{", "[" )
    new_string2 = new_string.replace("}", "]" )

    y = json.loads(new_string2)

    print(y)
    # print(type(y))

    B = np.array(y)
    print("B =", B)
    # print(B.shape)
else:
    B = []
    
# print(B)

[[0, 0, 1, -1, -1, 1], [1, -1, -1, 1, 0, 0], [1, -1, 0, 0, -1, 1]]
B = [[ 0  0  1 -1 -1  1]
 [ 1 -1 -1  1  0  0]
 [ 1 -1  0  0 -1  1]]


In [8]:
A,u = func(lst = G1)
u = u.to_numpy()
u = u.tolist()
u = np.array([item for items in u for item in items])

# u = u*20 + 1

A1 = A.flatten()

print("A =", A)
print("u =", u)
# print("u1 =", u1)

print(np.sum(u))

A = [[1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [1. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 1.]]
u = [ 73  87 309 355  28  65]
917


In [9]:
# RestingBP vs HeartDisease

A,u = func(lst = G1)
u = u.to_numpy()
u = u.tolist()
u = np.array([item for items in u for item in items])

Btemp = B
B = np.concatenate((Btemp,-Btemp))

# @jit(nopython=True, parallel=True, fastmath=True)
def calcs(numsamples, A, u, B):
    avert = np.array([A[:,i] for i in range(A.shape[1])])
    a = np.zeros(numsamples)
    a3 = np.zeros(numsamples)
    for i7 in prange(numsamples):
        a[i7] = np.random.rand()
        a3[i7] = np.random.rand()

    randlist1 = a.copy()
    randlist2 = a3.copy()
    chilist = np.zeros(numsamples)
    
    p1 = np.sum(u)*np.ones(u.shape[0])/u.shape[0]  
    
    tol1 = 0.001
    adot = np.zeros(A.shape[0])
    pcalc = np.zeros(A.shape[0])
    p1tempcalc = np.zeros(A.shape[0])
    ptempcalc = np.zeros(A.shape[0])

    
    adot1 = np.dot(A,u)

    for t in range(200):
        if np.sum(np.absolute(np.dot(A,p1) - adot1)) <= tol1:
            break
        else:
            p1temp = p1.copy()
            p1 = np.array([p1temp[j] * (np.prod(np.power(adot1, avert[j])) / np.prod(np.power(np.dot(A,p1temp), avert[j])))**0.5 for j in range(u.shape[0])])    
    
    chip = np.sum(np.array([((u[a1] - p1[a1])**2)/p1[a1] for a1 in range(u.shape[0])]))
    
    if G1 != [[i+1 for i in range(df.shape[1])]]:
        v = u
        for m in range(numsamples):
            w = B[int(len(B) * np.random.rand())]
            w1 = w.copy()

            tempmin = v[0] + w1[0]
            for j3 in prange(w1.shape[0]):
                if v[j3] + w1[j3] <= tempmin:
                    tempmin = v[j3] + w1[j3]

            if tempmin < 0:
                break
            else:       
                qprod = np.array([v[i] / (v[i] + w1[i]) if v[i] != v[i] + w1[i] and v[i] != 0 and (v[i] + w1[i]) != 0 else 1 for i in range(len(v))])

                q = np.prod(qprod)


                if q >= 1:
                    v = v + w1
                elif randlist2[m] <= q:
                    v = v + w1

            p = np.sum(u)*np.ones(u.shape[0])/u.shape[0]
            tol1 = 0.01
            adot = np.dot(A,v)

            for t2 in range(100):
                if np.sum(np.absolute(np.dot(A,p) - adot)) <= tol1:
                    break
                else:
                    ptemp = p.copy()
                    p = np.array([ptemp[j9] * (np.prod(np.power(adot, avert[j9])) / np.prod(np.power(np.dot(A,ptemp), avert[j9])))**0.5 for j9 in range(u.shape[0])])

            chitemp = np.array([((v[b1] - p[b1])**2)/p[b1] for b1 in range(u.shape[0])])
            if np.sum(chitemp) >= chip:
                chilist[m] = 1
    
    
        return np.sum(chilist)/numsamples, p1
    
    else:
        return 1, p1
    
pval, mleG = calcs(10000, A, np.array(u), B)

print("mleG =", mleG)
print("pval =", pval)



mleG = [ 71.53769724  88.4624516  296.88104063 367.11867589  41.58130804
  51.4188266 ]
pval = 0.074


In [10]:
np.sum([(u[i] - mleG[i])**2 / mleG[i] for i in range(len(u))])

8.971923226464275